In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
from itertools import permutations
from itertools import combinations
from random import choice
import time
import copy
from scipy.stats import norm,gumbel_r,expon,uniform
import cvxpy as cp

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
def bisection(prob,n,assortment,v,alpha,lb,ub):
    
    prob[assortment[0]] = (lb+ub)/2
    for j in range(1,len(assortment)):
        prob[assortment[j]] = math.exp(alpha[assortment[j]]*v[assortment[j]])* math.pow(prob[assortment[0]], alpha[assortment[j]]/alpha[assortment[0]]) 
    
    if np.abs(np.sum(prob)-1)< 0.000001:
        return prob
    elif sum(prob)>1:
        return bisection(prob,n,assortment,v,alpha,lb,prob[assortment[0]])
    else:
        return bisection(prob,n,assortment,v,alpha,prob[assortment[0]],ub)

In [4]:
def binarySearch(lb,ub,v,assortment,x,alpha):
    
    mid = (lb+ub)/2
    for j in assortment:
        x[j] = 1-expon.cdf(mid-v[j],scale=alpha[j])
    
    if np.abs(sum(x)-1)<0.000001:
        return x
    elif sum(x)>1:
        return binarySearch(mid,ub,v,assortment,x,alpha)
    else:
        return binarySearch(lb,mid,v,assortment,x,alpha)


In [5]:
def collection_distribution_mem(n,collection,v,alpha):
    
    collection_distribution = np.zeros((n,len(collection)))
    lb = -2000
    ub = 2000
    for i in range(len(collection)):
        prob = np.zeros(n)
        collection_distribution[:,i] = binarySearch(lb,ub,v,collection[i],prob,alpha)
        #collection_distribution[:,i] = bisection(prob,n,collection[i],v,alpha,lb,ub)

    return collection_distribution

In [6]:
''' ### product size = 7
n = 7
price = np.random.uniform(1,7,n)
instance_size = 2
collection_size = [20,40,80]

all_possible_assortment = []
for i in range(2,n):
    new_assortments = [list(x) for x in list(combinations(range(n),i))]
    all_possible_assortment.extend(new_assortments) '''

' ### product size = 7\nn = 7\nprice = np.random.uniform(1,7,n)\ninstance_size = 2\ncollection_size = [20,40,80]\n\nall_possible_assortment = []\nfor i in range(2,n):\n    new_assortments = [list(x) for x in list(combinations(range(n),i))]\n    all_possible_assortment.extend(new_assortments) '

In [7]:
### MDM prediction MILP MIN
def robust_mdm_revenue(data,collection,unseen_assortment,price):
    #print('The unseen assortment is',unseen_assortment)
    #print('the seen data is ',data )
    eps = math.pow(10,-6)
    l = len(unseen_assortment)
    n,m = data.shape
    
    ind = [[] for i in range(n)]  ## keeps tracking of the assortments including product i 
    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i) ## assortment i includes product j 
    
    ## MILP for revenue prediction
    x = cp.Variable(shape=l,nonneg = True) ## prob vector for unseen assortment
    delta_plus = cp.Variable(m,boolean=True)  ## indicator variables delta(S)+
    delta_minu = cp.Variable(m,boolean=True)  ## indicator variables delta(S)-
    #z = cp.Variable((l,m), boolean = True)  ## indicator variables z(i,A,S) 
    #y = cp.Variable(shape=l,boolean=True)  ## indicator variables y(i,A)  # indicator of sgn of x(i,A)
    eta = cp.Variable(1,nonneg=True)     ## lambda(A) 
    lam = cp.Variable(shape=m,nonneg=True)  ## lambda_S S stands for assortment
    
    
    ## normalization constraints of unseen assortment and bound on the lambda(A)
    
    constraints = [cp.sum(x)==1]
    constraints += [eta<=1]
        

    ### out-of-sample MDM consistency    
    for k in range(len(unseen_assortment)):
        #constraints += [x[k]-1+y[k]<=0]
        
        #for i in range(len(ind[unseen_assortment[k]])):
        for i in ind[unseen_assortment[k]]:
            #print('the comparison product',unseen_assortment[k])
            #print('the comparison assortment is',ind[unseen_assortment[k]][i])
            #print('the comparison data is ',data[unseen_assortment[k]][ind[unseen_assortment[k]][i]])
            constraints += [eta - lam[i] + delta_plus[i]>=0]
            constraints += [eta - lam[i] - 1 + delta_plus[i] + eps*delta_plus[i]<=0]
            constraints += [lam[i] - eta + delta_minu[i]>=0]
            constraints += [lam[i] - eta  - 1 + delta_minu[i] + eps*delta_minu[i]<=0]
            
            constraints += [x[k]-data[unseen_assortment[k]][i] - delta_plus[i] +1 >=0]
            constraints += [x[k]-data[unseen_assortment[k]][i] + delta_minu[i] -1 <=0]
            
            constraints += [x[k]-data[unseen_assortment[k]][i] + delta_plus[i] +delta_minu[i] >=0]
            constraints += [x[k]-data[unseen_assortment[k]][i] - delta_plus[i] -delta_minu[i]<=0]
              
      
    ## in-sample MDM constraints
    for i in range(n):
        if len(ind[i])>1: ### the comparison happens when there are at least two assortments including product i 
            for j in range(len(ind[i])-1):
                for k in range(j+1,len(ind[i])):
                    if data[i][ind[i][j]] < data[i][ind[i][k]]:
                        constraints += [lam[ind[i][j]] - lam[ind[i][k]] - eps >=0]
                    if data[i][ind[i][k]] < data[i][ind[i][j]]:
                        constraints += [lam[ind[i][k]] - lam[ind[i][j]] - eps >=0]
                    if data[i][ind[i][k]] == data[i][ind[i][j]]:
                        constraints += [lam[ind[i][k]] - lam[ind[i][j]] ==0]
                        
    ## bounds for lambda
    for i in range(m):
        constraints += [lam[i]<=1] 
        
    ### testing the true values
    ''' for i in range(len(unseen_assortment)):
        constraints += [x[i]==true_choice_prob[i]] '''
    
    ## objective  
    obj = cp.Minimize(sum(price[unseen_assortment[i]]*x[i] for i in range(len(unseen_assortment))))
    
    prob = cp.Problem(obj,constraints)
    prob.solve()     
    
    return [prob.value,x.value,lam.value,eta.value]


In [8]:
### MDM prediction MILP MIN
def ucb_mdm_revenue(data,collection,unseen_assortment,price):
    #print('The unseen assortment is',unseen_assortment)
    #print('the seen data is ',data )
    eps = math.pow(10,-6)
    l = len(unseen_assortment)
    n,m = data.shape
    
    ind = [[] for i in range(n)]  ## keeps tracking of the assortments including product i 
    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i) ## assortment i includes product j 
    
    ## MILP for revenue prediction
    x = cp.Variable(shape=l,nonneg = True) ## prob vector for unseen assortment
    delta_plus = cp.Variable(m,boolean=True)  ## indicator variables delta(S)+
    delta_minu = cp.Variable(m,boolean=True)  ## indicator variables delta(S)-
    #z = cp.Variable((l,m), boolean = True)  ## indicator variables z(i,A,S) 
    #y = cp.Variable(shape=l,boolean=True)  ## indicator variables y(i,A)  # indicator of sgn of x(i,A)
    eta = cp.Variable(1,nonneg=True)     ## lambda(A) 
    lam = cp.Variable(shape=m,nonneg=True)  ## lambda_S S stands for assortment
    
    
    ## normalization constraints of unseen assortment and bound on the lambda(A)
    
    constraints = [cp.sum(x)==1]
    constraints += [eta<=1]
        

    ### out-of-sample MDM consistency    
    for k in range(len(unseen_assortment)):
        #constraints += [x[k]-1+y[k]<=0]
        
        #for i in range(len(ind[unseen_assortment[k]])):
        for i in ind[unseen_assortment[k]]:
            #print('the comparison product',unseen_assortment[k])
            #print('the comparison assortment is',ind[unseen_assortment[k]][i])
            #print('the comparison data is ',data[unseen_assortment[k]][ind[unseen_assortment[k]][i]])
            constraints += [eta - lam[i] + delta_plus[i]>=0]
            constraints += [eta - lam[i] - 1 + delta_plus[i] + eps*delta_plus[i]<=0]
            constraints += [lam[i] - eta + delta_minu[i]>=0]
            constraints += [lam[i] - eta  - 1 + delta_minu[i] + eps*delta_minu[i]<=0]
            
            constraints += [x[k]-data[unseen_assortment[k]][i] - delta_plus[i] +1 >=0]
            constraints += [x[k]-data[unseen_assortment[k]][i] + delta_minu[i] -1 <=0]
            
            constraints += [x[k]-data[unseen_assortment[k]][i] + delta_plus[i] +delta_minu[i] >=0]
            constraints += [x[k]-data[unseen_assortment[k]][i] - delta_plus[i] -delta_minu[i]<=0]
              
      
    ## in-sample MDM constraints
    for i in range(n):
        if len(ind[i])>1: ### the comparison happens when there are at least two assortments including product i 
            for j in range(len(ind[i])-1):
                for k in range(j+1,len(ind[i])):
                    if data[i][ind[i][j]] < data[i][ind[i][k]]:
                        constraints += [lam[ind[i][j]] - lam[ind[i][k]] - eps >=0]
                    if data[i][ind[i][k]] < data[i][ind[i][j]]:
                        constraints += [lam[ind[i][k]] - lam[ind[i][j]] - eps >=0]
                    if data[i][ind[i][k]] == data[i][ind[i][j]]:
                        constraints += [lam[ind[i][k]] - lam[ind[i][j]] ==0]
                        
    ## bounds for lambda
    for i in range(m):
        constraints += [lam[i]<=1] 
        
    ### testing the true values
    ''' for i in range(len(unseen_assortment)):
        constraints += [x[i]==true_choice_prob[i]] '''
    
    ## objective  
    obj = cp.Maximize(sum(price[unseen_assortment[i]]*x[i] for i in range(len(unseen_assortment))))
    
    prob = cp.Problem(obj,constraints)
    prob.solve()     
    
    return [prob.value,x.value,lam.value,eta.value]


In [9]:
## MNL MLE 
def mnl_mle(data,collection):
    binary_collection = [] ### create collection zero-one matrix

    for i in range(len(collection)):
        temp = [-9999999999999999999]*data.shape[0]
        for j in collection[i]:
            temp[j] = 1
        binary_collection.append(temp)
        
    n = data.shape[0]
    v= cp.Variable(n)
    obj = 0
    for i in range(len(collection)): 
        for j in collection[i]:
            obj = obj + data[j][i]*v[j]  
        obj = obj -  cp.log_sum_exp(cp.multiply(binary_collection[i],v))  
         
    prob = cp.Problem(cp.Maximize(obj))
    prob.solve()

    return v.value


In [10]:
### product size = 7
n = 7
price = np.random.uniform(1,7,n)
instance_size = 20
collection_size = [20,40,80]
#collection_size = [2,3,5,10,15,20]
#collection_size = [80]

all_possible_assortment = []
for i in range(2,n):
    new_assortments = [list(x) for x in list(combinations(range(n),i))]
    all_possible_assortment.extend(new_assortments)

''' all_size2_ass = [list(x) for x in list(combinations(range(n),2))]
all_size3_ass = [list(x) for x in list(combinations(range(n),3))]
all_possible_assortment = all_size2_ass + all_size3_ass  '''

' all_size2_ass = [list(x) for x in list(combinations(range(n),2))]\nall_size3_ass = [list(x) for x in list(combinations(range(n),3))]\nall_possible_assortment = all_size2_ass + all_size3_ass  '

In [11]:
avg_min_max_interval = []
avg_min_true_interval = []
avg_max_true_interval =[]
all_true_revenue = []
all_min_revenue = []
all_max_revenue = []

all_mnl_revenue_list = []

for i in range(len(collection_size)):
    instances =[]
    true_mem_revenue_list = []
    
    lb_mdm_revenue_list = []
    ub_mdm_revenue_list = []
    min_max_interval = []
    min_true_interval =[]
    max_true_interval =[]
    
    mnl_revenue_list =[]
    
    for j in range(instance_size):
        
        # generate preference parameters 
        v = np.random.uniform(-5,5,n)
        
        ## case 1
        alpha = np.random.uniform(1,2,n)
        
        ## case 2
        #alpha = np.random.uniform(10,20,n)
                
        ## case 3
        #alpha = np.random.uniform(1,20,n)
        
        # generate assortment collection for the current instance
        assortment_collection = []
        chosen_idx = random.sample(range(len(all_possible_assortment)),collection_size[i])
        chosen_idx.sort()
        for k in chosen_idx:
            assortment_collection.append(all_possible_assortment[k])
        
        # generate instance - observed choice probabilities
        curr_instance = collection_distribution_mem(n,assortment_collection,v,alpha)
        instances.append(curr_instance)
            
        ''' # generate unseen assortment 
        unseen_set = [x for x in range(len(all_possible_assortment)) if x not in chosen_idx]
        unseen_idx = random.sample(unseen_set,1)
        unseen_assortment = all_possible_assortment[unseen_idx[0]] '''
        
        # generate unseen assortment 
        ''' len_S = 2
        if np.random.random()>0.5:
            len_S = 3 '''
        
        available  = np.unique(curr_instance.nonzero()[0])
        available = list(available)
        len_S = random.randint(2,len(available))
        unseen_assortment = random.sample(available,len_S) 
        unseen_assortment.sort()
        
        # compute true revenue 
        prob =np.zeros(n)
        lb = -2000
        ub = 2000
        true_choice_prob = binarySearch(lb,ub,v,unseen_assortment,prob,alpha)
        
        true_revenue = np.sum(price*true_choice_prob) 
            
        true_mem_revenue_list.append(true_revenue)
            
        # compute robust mdm revenue
        mdm_revenue = robust_mdm_revenue(curr_instance,assortment_collection,unseen_assortment,price)
        #print('robust mdm revenue',mdm_revenue[0])
        lb_mdm_revenue_list.append(mdm_revenue[0])
        
        if mdm_revenue[0] - true_revenue>0.001:
            print('error in the LB')
            df = pd.DataFrame(curr_instance)
            df.to_csv('instance'+str(i)+'.csv')
            print('ground truth revenue',true_revenue)
            print('the unseen assortment is',unseen_assortment)
            print('the ground trutch choice probabilities',true_choice_prob)
            print('the mdm solution is',mdm_revenue[0],mdm_revenue[1])
            
        min_true_interval.append(true_revenue - mdm_revenue[0])
        
        mdm_revenue_ub = ucb_mdm_revenue(curr_instance,assortment_collection,unseen_assortment,price)
        #print('ucb mdm revenue',mdm_revenue_ub[0])
        ub_mdm_revenue_list.append(mdm_revenue_ub[0])
        
        if true_revenue - mdm_revenue_ub[0] > 0.001:
            print('error in the UB')
            df = pd.DataFrame(curr_instance)
            df.to_csv('instance'+str(i)+'.csv')
            print('ground truth revenue',true_revenue)
            print('the unseen assortment is',unseen_assortment)
            print('the ground trutch choice probabilities',true_choice_prob)
            print('the mdm solution is',mdm_revenue_ub[0],mdm_revenue_ub[1])
        
        max_true_interval.append(mdm_revenue_ub[0] - true_revenue)
        min_max_interval.append(mdm_revenue_ub[0] - mdm_revenue[0])
    
        ## compute mle of the parameters
        mle_parameter = mnl_mle(curr_instance,assortment_collection)
        
        # compute choice probability of the unseen assortment using mle
        unseen_prob = []
        total_weight = 0
        for k in unseen_assortment:
            total_weight = total_weight + math.exp(mle_parameter[k])
        for k in unseen_assortment:
            unseen_prob.append(math.exp(mle_parameter[k])/total_weight)
        #mle_distribution = mnl_ditribution(mle_parameter,unseen_assortment)
        mle_revenue = 0
        for k in range(len(unseen_assortment)):
            mle_revenue = mle_revenue + price[unseen_assortment[k]]*unseen_prob[k]
        mnl_revenue_list.append(mle_revenue)
    
      
    df = pd.DataFrame({'true_revenue':true_mem_revenue_list,'min_mdm_revenue':lb_mdm_revenue_list,'max_mdm_revenue':ub_mdm_revenue_list,'mnl_revenue':mnl_revenue_list})
    df.to_csv('MEM/case1/'+str(collection_size[i])+'/revenues_withMNL.csv')
        
    all_true_revenue.append(true_mem_revenue_list)
    all_min_revenue.append(lb_mdm_revenue_list)
    all_max_revenue.append(ub_mdm_revenue_list)
    
    all_true_revenue.append(true_mem_revenue_list)
    all_mnl_revenue_list.append(mnl_revenue_list)
    
    avg_min_max_interval.append(np.mean(min_max_interval))
    avg_min_true_interval.append(np.mean(min_true_interval))   
    avg_max_true_interval.append(np.mean(max_true_interval)) 
    
    

In [12]:
summary = pd.DataFrame({'assortment size':collection_size,'avg_min_max':avg_min_max_interval,'avg_min_true':avg_min_true_interval,'avg_max_true_interval':avg_max_true_interval})
summary.to_csv('MEM/case1/summary_2.csv')

In [13]:
summary

,assortment size,avg_min_max,avg_min_true,avg_max_true_interval
0,20,0.092881,0.057990,0.034892
1,40,0.062389,0.029201,0.033188
2,80,0.012516,0.006789,0.005727


In [14]:
## MNL MLE 
def mnl_mle(data,collection):
    binary_collection = [] ### create collection zero-one matrix

    for i in range(len(collection)):
        temp = [-9999999999999999999]*data.shape[0]
        for j in collection[i]:
            temp[j] = 1
        binary_collection.append(temp)
        
    n = data.shape[0]
    v= cp.Variable(n)
    obj = 0
    for i in range(len(collection)): 
        for j in collection[i]:
            obj = obj + data[j][i]*v[j]  
        obj = obj -  cp.log_sum_exp(cp.multiply(binary_collection[i],v))  
         
    prob = cp.Problem(cp.Maximize(obj))
    prob.solve()

    return v.value


In [15]:
mle_prediction_performance = [[] for _ in range(len(all_mnl_revenue_list))]
for i in range(len(all_mnl_revenue_list)):
    for j in range(instance_size):
        indicator = 0
        if all_mnl_revenue_list[i][j]>=all_min_revenue[i][j] and all_mnl_revenue_list[i][j]<=all_max_revenue[i][j]:
            indicator = 1
        mle_prediction_performance[i].append(indicator)

In [16]:
performance = np.array(mle_prediction_performance)
avg_performance = np.mean(performance,axis=1)
avg_performance

array([0.1, 0. , 0. ])